# Datasets

** Local and Global Algorithms ... **

AQUAINT: Milne

MSNBC dataset, taken from (Cucerzan, 2007),

ACE: Mechanical Turkn

Wiki: choose those paragraphts that p(t|m) makes atleast 10% error

For evaluation, check BOT evaluation, mentioned in Milne 

Downloadable from :
http://cogcomp.cs.illinois.edu/page/resource_view/4


**Spotlight**
two datasets, a wiki selection
35 paragraphs from New York times
There is a website, but couldn't find it

http://oldwiki.dbpedia.org/Datasets/NLP

Tag me:
Wiki and tweet, 
available, but looks old!
http://acube.di.unipi.it/tagme-dataset/

**AIDA**

https://www.mpi-inf.mpg.de/departments/databases-and-information-systems/research/yago-naga/aida/downloads/
AIDA CoNLL-YAGO Dataset: Hnad create from Conll
AIDA-EE Dataset: Again hand done


In [ ]:
%%writefile config.py 
""" Config file.
"""
#%load_ext autoreload
#%autoreload

import os.path
import sys
import datetime


home = os.path.expanduser("~");
dsdir_ner = "../datasets/ner";




In [ ]:
from config import *
get_wikifify_params(0)

# Simple Test

In [ ]:
# %load_ext autoreload
# %autoreload

# %aimport wsd
# import sys
from wikify import *
import time
ws=5
S=["Carlos", "met", "David", "and" , "Victoria", "in", "Madrid"]
M=[[0, "Roberto_Carlos"], [2, "David_Beckham"], [4, "Victoria_Beckham"], [6, "Madrid"]]

S=["David", "met", "Victoria", "and" , "Victoria", "in", "Madrid"]
M=[[0, "David_Beckham"], [2, "Victoria_Beckham"], [6, "Madrid"]]
S=["Phoenix, Arizona"] 
M=[[0, "Phoenix,_Arizona"]]

start = time.time()
C = generate_candidates(S, M, max_t=5, enforce=False)
print "Candidates: ", C, "\n"

ids, titles = wikify(S,M,C, ws, method='context2context')


#print "Key Scores_method_1: ", candslist_scores, "\n"
print "Best IDS", ids, "\n"
print "Best titles", titles, "\n"
#print "get_tp",get_tp(M, ids) 






# Legacy WSD Evaluation

In [ ]:
%%writefile wsd_legacy_eval.py 
'''The Legacy WSD evaluation
This is how we evaluated for the thesis. 
The main difference is "enforce=True" in the candidate generation, 
an option which always makes sure the correct entity is among the candidates
'''


import sys
from optparse import OptionParser

#sys.path.insert(0,'..')
#from wikisim.calcsim import *
from wsd import *
import time
from random import shuffle
from config import *
np.seterr(all='raise')


parser = OptionParser()
parser.add_option("-t", "--max_t", action="store", type="int", dest="max_t", default=5)
parser.add_option("-c", "--max_count", action="store", type="int", dest="max_count", default=-1)
parser.add_option("-v", action="store_true", dest="verbose", default=False)

(options, args) = parser.parse_args()



max_t = options.max_t
max_count = options.max_count
verbose = options.verbose
# ws = options.win_size


# max_t = 15
# max_count = 20
# verbose = True

fresh_restart=False



dsnames = [os.path.join(dsdir_ner, 'kore.json'),
          os.path.join(dsdir_ner, 'wiki-mentions.5000.json'),
          os.path.join(dsdir_ner, 'aida.json'),  
          os.path.join(dsdir_ner, 'msnbc.txt.json'),
          os.path.join(dsdir_ner, 'aquaint.txt.json') 
          ]

methods = ['popularity','keydisamb','entitycontext','mention2entity','context2context','context2profile', 'learned']
methods = ['popularity','mention2entity','context2context','context2profile', 'learned']
methods = ['context2context']

outdir = os.path.join(baseresdir, 'wsd_legacy')
# if not os.path.exists(outdir): #Causes synchronization problem
#     os.makedirs(outdir)

tmpdir = os.path.join(outdir, 'tmp')
# if not os.path.exists(tmpdir): #Causes synchronization problem
#     os.makedirs(tmpdir)
    
resname =  os.path.join(outdir, 'reslog.csv')
#clearlog(resname)

detailedresname=  os.path.join(outdir, 'detailedreslog.txt')
#clearlog(detailedresname)



for method in methods:
    if 'word2vec' in method:
        gensim_loadmodel(word2vec_path)
        print "loaded"
        sys.stdout.flush()
    for dsname in dsnames:
        start = time.time()
        
        print "dsname: %s, method: %s, op_method: %s, direction: %s, max_t: %s, ws: %s ..."  % (dsname,
                "rvspagerank", method, "both", max_t, 5)
        
        sys.stdout.flush()
        
        tmpfilename = os.path.join(tmpdir, 
                                   '-'.join(['rvspagerank', str(DIR_BOTH), method, str(max_t), '5', os.path.basename(dsname)]))
        
        overall=[]
        start_count=-1
        if os.path.isfile(tmpfilename) and not fresh_restart:
            with open(tmpfilename,'r') as tmpf:
                for line in tmpf:
                    js = json.loads(line.strip())
                    start_count = js['no']
                    if js['tp'] is not None:
                        overall.append(js['tp'])
        
        if start_count !=-1:
            print "Continuing from\t", start_count
            
        count=0
        with open(dsname,'r') as ds, open(tmpfilename,'a') as tmpf:
            for line in ds:
                if (max_count !=-1) and (count >= max_count):
                    break
                
                js = json.loads(line.decode('utf-8').strip());
                S = js["text"]
                M = js["mentions"]
                count +=1
                if count <= start_count:
                    continue
                    
                if verbose:
                    print "%s:\tS=%s\n\tM=%s" % (count, json.dumps(S, ensure_ascii=False).encode('utf-8'),json.dumps(M, ensure_ascii=False).encode('utf-8'))
                    sys.stdout.flush()
                    
                C = generate_candidates(S, M, max_t=max_t, enforce=True)
                
                try:
                    #ids, titles = disambiguate_driver(S,M, C, ws=0, method=method, direction=direction, op_method=op_method)
                    ids, titles = wsd(S,M,C, method=method)
                    tp = get_tp(ids, M) 
                except Exception as ex:
                    tp = (None, None)
                    print "[Error]:\t", type(ex), ex
                    raise
                    continue
                
                overall.append(tp)
                tmpf.write(json.dumps({"no":count, "tp":tp})+"\n")
                    

        elapsed = str(timeformat(int(time.time()-start)));
        print "done"
        
        detailedres ={"dsname":dsname, "method": "rvspagerank", "op_method": method, "driection": 'both',
                      "max_t": max_t, "tp":overall, "elapsed": elapsed, "ws": 5}
        
        
        logres(detailedresname, '%s',  json.dumps(detailedres))
        
        micro_prec, macro_prec = get_prec(overall)        
        logres(resname, '%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s', 'rvspagerank', method, 'both', max_t , 5, 
               dsname, micro_prec, macro_prec, elapsed)
        

print "done"

# WSD Evaluation

In [ ]:
%%writefile wsd_new_eval.py 
''' The new WSD evaluation method, here some of the options we already settled on
    is fixed and hidden, such as similarity method or ws
'''
import sys
from optparse import OptionParser

#sys.path.insert(0,'..')
#from wikisim.calcsim import *
from wsd import *
import time
from random import shuffle
from config import *

np.seterr(all='raise')

parser = OptionParser()
parser.add_option("-t", "--max_t", action="store", type="int", dest="max_t", default=5)
parser.add_option("-c", "--max_count", action="store", type="int", dest="max_count", default=-1)
parser.add_option("-v", action="store_true", dest="verbose", default=False)

(options, args) = parser.parse_args()



max_t = options.max_t
max_count = options.max_count
verbose = options.verbose


# max_t = 15
# max_count = 20
# verbose = True

fresh_restart=False



dsnames = [os.path.join(dsdir_ner, 'kore.json'),
          os.path.join(dsdir_ner, 'wiki-mentions.5000.json'),
#          os.path.join(dsdir_ner, 'aida.json'),  
          os.path.join(dsdir_ner, 'msnbc.txt.json'),
          os.path.join(dsdir_ner, 'aquaint.txt.json') 
          ]

methods = ['popularity','keydisamb','entitycontext','mention2entity','context2context','context2profile', 'learned']

outdir = os.path.join(baseresdir, 'wsd_new')
# if not os.path.exists(outdir): #Causes synchronization problem
#     os.makedirs(outdir)

tmpdir = os.path.join(outdir, 'tmp')
# if not os.path.exists(tmpdir): #Causes synchronization problem
#     os.makedirs(tmpdir)
    
resname =  os.path.join(outdir, 'reslog.csv')
#clearlog(resname)

detailedresname=  os.path.join(outdir, 'detailedreslog.txt')
#clearlog(detailedresname)

ltr_nrows=10000
load_wsd_model(ltr_nrows)


for method in methods:
    if 'word2vec' in method:
        gensim_loadmodel(word2vec_path)
        print "loaded"
        sys.stdout.flush()
    for dsname in dsnames:
        start = time.time()
        
        print "dsname: %s, method: %s, max_t: %s ..."  % (dsname,
                method, max_t)
        sys.stdout.flush()
        
        tmpfilename = os.path.join(tmpdir, 
                                   '-'.join([method, str(max_t), os.path.basename(dsname)]))
        overall=[]
        start_count=-1
        if os.path.isfile(tmpfilename) and not fresh_restart:
            with open(tmpfilename,'r') as tmpf:
                for line in tmpf:
                    js = json.loads(line.strip())
                    start_count = js['no']
                    if js['tp'] is not None:
                        overall.append(js['tp'])
        
        if start_count !=-1:
            print "Continuing from\t", start_count
            
        count=0
        with open(dsname,'r') as ds, open(tmpfilename,'a') as tmpf:
            for line in ds:
                if (max_count !=-1) and (count >= max_count):
                    break
                js = json.loads(line.decode('utf-8').strip());
                S = js["text"]
                M = js["mentions"]
                count +=1
                if count <= start_count:
                    continue
                if verbose:
                    print "%s:\tS=%s\n\tM=%s" % (count, json.dumps(S, ensure_ascii=False).encode('utf-8'),json.dumps(M, ensure_ascii=False).encode('utf-8'))
                    sys.stdout.flush()
                    
                C = generate_candidates(S, M, max_t=max_t, enforce=False)
                
                try:
                    #ids, titles = disambiguate_driver(S,M, C, ws=0, method=method, direction=direction, op_method=op_method)
                    ids, titles = wsd(S,M,C, method=method)
                    tp = get_tp(ids, M) 
                except Exception as ex:
                    tp = (None, None)
                    print "[Error]:\t", type(ex), ex
                    raise
                    continue
                
                overall.append(tp)
                tmpf.write(json.dumps({"no":count, "tp":tp})+"\n")
                    

        elapsed = str(timeformat(int(time.time()-start)));
        print "done"
        detailedres ={"dsname":dsname, "method": method, 
                      "max_t": max_t, "tp":overall, "elapsed": elapsed, "problem": "wsd"}
        
        
        logres(detailedresname, '%s',  json.dumps(detailedres))
        
        micro_prec, macro_prec = get_prec(overall)        
        logres(resname, 'wikify_link\t%s\t%s\t%s\t%s\t%s\t%s', method, max_t, 
               dsname, micro_prec, macro_prec, elapsed)

print "done"

# Wikification Evaluation

In [ ]:
%%writefile wikify_eval.py
"""Evaluating the wsd module. It assumes the sentences are already segmented
"""
from wikify import *
import sys
from optparse import OptionParser
#sys.path.insert(0,'..')
#from wikisim.calcsim import *
import time
from random import shuffle
from config import *

np.seterr(all='raise')

parser = OptionParser()
parser.add_option("-t", "--max_t", action="store", type="int", dest="max_t", default=5)
parser.add_option("-c", "--max_count", action="store", type="int", dest="max_count", default=-1)
parser.add_option("-v", action="store_true", dest="verbose", default=False)

(options, args) = parser.parse_args()



max_t = options.max_t
max_count = options.max_count
verbose = options.verbose


# max_t = 20
# max_count = 30
# verbose = True

fresh_restart=False


dsnames = [os.path.join(dsdir_ner, 'kore.json'),
          os.path.join(dsdir_ner, 'wiki-mentions.5000.json'),
#          os.path.join(dsdir_ner, 'aida.json'),  
          os.path.join(dsdir_ner, 'msnbc.txt.json'),
          os.path.join(dsdir_ner, 'aquaint.txt.json') 
          ]



mentionmethods = (LEARNED_MENTION, CORE_NLP)
mentionmethods = (LEARNED_MENTION,)

outdir = os.path.join(baseresdir, 'wikify')
# if not os.path.exists(outdir): #Causes synchronization problem
#     os.makedirs(outdir)

tmpdir = os.path.join(outdir, 'tmp')
# if not os.path.exists(tmpdir): #Causes synchronization problem
#     os.makedirs(tmpdir)
    
resname =  os.path.join(outdir, 'reslog.csv')
#clearlog(resname)

detailedresname=  os.path.join(outdir, 'detailedreslog.txt')
#clearlog(detailedresname)


svc_params = ((SVC_HP_NROWS_S, SVC_HP_CV_S), 
              (SVC_HR_NROWS_S, SVC_HR_CV_S))
svc_params = ((SVC_HP_NROWS_L, SVC_HP_CV_L), 
              (SVC_HR_NROWS_L, SVC_HR_CV_L))


ltr_nrows=LTR_NROWS_L

load_wsd_model(ltr_nrows)


for mentionmethod in mentionmethods:
    if mentionmethod == LEARNED_MENTION:
        for svc_nrows, svc_cv in svc_params:
            load_mention_model(svc_nrows, svc_cv)
            
            for dsname in dsnames:
                start = time.time()

                print "dsname: %s, mentionmethods: %s, max_t: %s ..."  % (dsname,
                        mentionmethods, max_t)
                sys.stdout.flush()

                tmpfilename = os.path.join(tmpdir, 
                                           '-'.join([str(mentionmethod), str(max_t), str(svc_nrows), str(svc_cv), str(ltr_nrows) ,os.path.basename(dsname)]))
                mention_overall=[]
                wikify_overall=[]
                start_count=-1
                if os.path.isfile(tmpfilename) and not fresh_restart:
                    with open(tmpfilename,'r') as tmpf:
                        for line in tmpf:
                            js = json.loads(line.strip())
                            start_count = js['no']

                            if js['mention_measures'] is not None:
                                mention_overall.append(js['mention_measures'])

                            if js['wikify_measures'] is not None:
                                wikify_overall.append(js['wikify_measures'])
                if start_count !=-1:
                    print "Continuing from\t", start_count

                count=0
                with open(dsname,'r') as ds, open(tmpfilename,'a') as tmpf:
                    for line in ds:
                        if (max_count !=-1) and (count >= max_count):
                            break

                        js = json.loads(line.decode('utf-8').strip());
                        S = js["text"]
                        M = js["mentions"]
                        count +=1
                        if count <= start_count:
                            continue
                        if verbose:
                            print "%s:\tS=%s\n\tM=%s" % (count, json.dumps(S, ensure_ascii=False).encode('utf-8'),json.dumps(M, ensure_ascii=False).encode('utf-8'))
                            sys.stdout.flush()
                        text= " ".join(S)    
                        #S2,M2 = detect_mentions(text, mentionmethod=mentionmethod)      
                        S2,M2 = wikify_string(text, mentionmethod=mentionmethod)

                        mention_measures = get_sentence_measures(S2, M2, S, M, wsd_measure=False)
                        mention_overall.append(mention_measures)

                        wikify_measures = get_sentence_measures(S2, M2, S, M, wsd_measure=True)
                        wikify_overall.append(wikify_measures)

                        tmpf.write(json.dumps({"no":count, "mention_measures":mention_measures, "wikify_measures":wikify_measures})+"\n")



                elapsed = str(timeformat(int(time.time()-start)));
                print "done"
                detailedres ={"mentionmethod": mentionmethod, "max_t":max_t, "svc_nrows": svc_nrows, "svc_cv": svc_cv, "ltr_nrows":ltr_nrows, "dsname": dsname,
                              "max_t": max_t, "mention_overall":mention_overall, "wikify_overall": wikify_overall, "elapsed": elapsed}


                logres(detailedresname, '%s',  json.dumps(detailedres))
                #print mention_overall

                mention_overall_measures = get_overall_measures(mention_overall)    
                output = ('mention_evaluation',mentionmethod, max_t, svc_nrows, svc_cv, ltr_nrows, dsname) + mention_overall_measures + (elapsed,)        
                logres(resname, '%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n', *output)

                #print wikify_overall
                wikify_overall_measures = get_overall_measures(wikify_overall)  
                output = ('wikify_evaluation',mentionmethod, max_t,  svc_nrows, svc_cv, ltr_nrows, dsname) + wikify_overall_measures + (elapsed,)
                logres(resname, '%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n', *output)
            

print "done"

In [ ]:
from wikify import *

svc_nrows, svc_cv = SVC_MODEL_HIGH_RECALL_NROWS, SVC_MODEL_HIGH_RECALL_CV

load_mention_model(svc_nrows, svc_cv)

ltr_nrows=10000
load_wsd_model(ltr_nrows)

S=["David", "and", "Victoria", "named", "their", "children", "Brooklyn", ",", "Romeo", ",", "Cruz", ",", "and", "Harper Seven", "."]
M=[[0, "David_Beckham"], [2, "Victoria_Beckham"]]

S=["Nixon", "resigned", "after", "Watergate", "despite", "his", "success", "in", "the", "Ping-Pong Diplomacy", "with", "China", "."]
M=[[0, "Richard_Nixon"], [3, "Watergate_scandal"], [9, "Ping_Pong_Diplomacy"], [11, "People's_Republic_of_China"]]
text = " ".join(S).decode('utf-8')
# print text
#S, M, _ = annotate_with_corenlp(text) 
S2,M2 = detect_mentions(text, mentionmethod=LEARNED_MENTION)      
#print M2
S2,M2 = wikify_string(text, mentionmethod=LEARNED_MENTION)
import cProfile as profile
profile.run('wikify_string(text, mentionmethod=LEARNED_MENTION)')

In [1]:
# %load problem
S=["John", "I", "of", "Antioch", "was", "Patriarch of Antioch", "(429–441)", "and", "led", "a", "group", "of", "moderate", "Eastern", "bishops", "during", "the", "Nestorian", "controversy.", "He", "is", "sometimes", "confused", "with", "John Chrysostom", ",", "who", "is", "occasionally", "also", "referred", "to", "as", "\"John", "of", "Antioch\".", "John", "gave", "active", "support", "to", "his", "friend", "Nestorius", "in", "the", "latter's", "dispute", "with", "Cyril of Alexandria", ".", "In", "the", "year", "431,", "he", "arrived", "too", "late", "for", "the", "opening", "meeting", "of", "the", "First Council of Ephesus", ".", "Cyril,", "suspecting", "John", "of", "using", "procrastinating", "tactics", "to", "support", "Nestorius,", "decided", "not", "to", "wait", "and", "convened", "the", "council", "without", "John", "and", "his", "supporters,", "condemning", "Nestorius.", "When", "John", "reached", "Ephesus", "a", "few", "days", "after", "the", "council", "had", "begun,", "he", "convened", "a", "counter-council", "which", "condemned", "Cyril", "and", "vindicated", "Nestorius."]
M=[[5, "List of Patriarchs of Antioch"], [17, "Nestorianism"], [24, "John Chrysostom"], [43, "Nestorius"], [49, "Cyril of Alexandria"], [65, "First Council of Ephesus"], [95, "Ephesus"]]
from wikify import *

svc_nrows, svc_cv = SVC_HP_NROWS_S, SVC_HP_CV_S


load_mention_model(svc_nrows, svc_cv)

ltr_nrows=LTR_NROWS_S
load_wsd_model(ltr_nrows)

text = " ".join(S).decode('utf-8')
# print text
#S, M, _ = annotate_with_corenlp(text) 
S2,M2 = detect_mentions(text, mentionmethod=LEARNED_MENTION)      
#print M2
S2,M2 = wikify_string(text, mentionmethod=LEARNED_MENTION)
S2
M2

solr encoding!


/users/grad/sajadi/backup/anaconda2/envs/wikisim/lib/python2.7/site-packages/scipy/spatial/distance.py:505: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))


solr encoding!


[[0, 'John_of_Antioch'],
 [8, 'Byzantine_Empire'],
 [17, 'John_Chrysostom'],
 [26, 'John_of_Antioch'],
 [33, 'Nestorius'],
 [52, 'Council_of_Ephesus'],
 [64, 'Nestorius'],
 [79, 'Nestorius'],
 [84, 'Council_of_Ephesus'],
 [102, 'Nestorius']]

In [ ]:
from wikify import *

svc_nrows, svc_cv = SVC_MODEL_HIGH_PRECISION_NROWS, SVC_MODEL_HIGH_PRECISION_CV

In [ ]:
print svc_nrows, svc_cv

In [3]:
type("is istr") is str

True